## Withdrawal precompile reentrancy tests

### Scenario A: `TokenProxyTester` withdraws tickets twice during withdrawal, but it has these tickets
- setup:
    - tester originated with callsCount=0, amount=10
    - deposit 1000 tickets to the proxy from L1 via kernel
    - tester resetup to have callsCount=1, amount=10
- test:
    - user calls `fa_withdrawal_precompile` to withdraw 1 ticket from the tester
        * tester withdraws 10 more tickets in the internal call
    - check transaction succeed
    - check that 989 tickets left on tester
    - check that 2 withdrawal events created, each for 2 tickets
    - check that 2 outbox messages created
    - deposit 21 tickets to the proxy from L1 via kernel
        * tester withdraws 10 more tickets in the internal call
    - check that 1 outbox message created
    - check that 1 deposit event created
    - check that 1000 tickets left on tester
    - user tries to withdraw > 990 tickets from tester and transaction fails
    - user tries tp withdraw 990 tickets and succeed
        * tester withdraws 10 more tickets in the internal call
    - check that two withdrawal events and 2 outbox messages created
    - deposit 15 tickets to the proxy from L1 via kernel
        * tester tries to withdraw 10 more tickets in the internal call but not succeed
    - check deposit succeed, deposit event created and tester recieves 15 tickets

In [1]:
from docs.scenarios.setup import *

accounts = setup()
web3, etherlink_account, tezos_account = accounts
_, ticketer, _, _, ticket_router_tester = load_contracts(*accounts)

Setup:
- Tezos account: `tz1ekkzEN2LB1cpf7dCaonKt6x9KVd9YVydc`, balance: `100.376908 ꜩ`
- Etherlink account: `0x7e6f6CCFe485a087F0F819eaBfDBfb1a49b97677`, balance: `1.661226239 ꜩ`


### Deploy TokenProxyTester:
- This is the contract that can be used in the replacement of the `ERC20Proxy` makes reentrant calls to the `fa_withdrawal_precompile` during both `deposit` and `withdrawal` calls.

In [6]:
calls_count = 1
withdraw_amount = 10
routing_info = make_withdrawal_routing_info(tezos_account, ticketer)
ticketer_address_bytes = tezos_address_to_bytes(ticketer.address)
# TODO: consider adding `to_bytes` method
ticket_content_bytes = bytes.fromhex(ticketer.read_content().to_bytes_hex())

token_proxy_tester = TokenProxyTesterHelper.originate_from_file(
    web3=web3,
    filename=make_filename('TokenProxyTester'),
    account=etherlink_account,
    constructor_args=(
        # TODO: the kernel address added as an initial Ticket Owner
        # it should be replaced with the contract that will be used as an ERC20Proxy:
        KERNEL_ADDRESS,
        FA_WITHDRAWAL_PRECOMPILE,
        routing_info,
        withdraw_amount,
        ticketer_address_bytes,
        ticket_content_bytes,
        calls_count,
    )
)

In [7]:
token_proxy_tester.address

'0x8f31Fa683c8DEdbE61fC772b2E0a29C35069FF9D'

### Work in progress ...
---

- Reload TokenProxyTester

In [3]:
token_proxy_tester = TokenProxyTesterHelper.from_address(
    web3,
    etherlink_account,
    '0x8f31Fa683c8DEdbE61fC772b2E0a29C35069FF9D'
)
token_proxy_tester.address

'0x8f31Fa683c8DEdbE61fC772b2E0a29C35069FF9D'

In [4]:
opg_hash = setup_ticket_router_tester_to_rollup_deposit(
    ticket_router_tester=ticket_router_tester,
    target=SMART_ROLLUP_ADDRESS,
    receiver=token_proxy_tester,
    # TODO: make no router (optional)
    router=token_proxy_tester,
)

Setting up TicketRouterTester to the rollupDeposit mode:
  - Target: `sr1G9N4WzRF79GeGKVd1vUDUg2NSrFGNGAfA`
  - Routing Info: `0x8f31fa683c8dedbe61fc772b2e0a29c35069ff9d8f31fa683c8dedbe61fc772b2e0a29c35069ff9d`
Successfully set, tx hash: `ooFL125t1MPdT5cAyuigss51HQue57NDwdTBkGGHGenEhReYPBs`


'ooFL125t1MPdT5cAyuigss51HQue57NDwdTBkGGHGenEhReYPBs'

In [5]:
ticket = ticketer.read_ticket(tezos_account)
ticket, _ = ticket.split(1_000)

opg_hash = transfer_ticket(
    ticket=ticket,
    receiver=ticket_router_tester,
)

Transfering ticket:
  - Owner: `tz1ekkzEN2LB1cpf7dCaonKt6x9KVd9YVydc`
  - Ticketer address: `KT1TQiSYR6VhFHso5xkEYR3Fgg5Vsz8iRaK7`
  - Ticket content: `0x0707000005090a000000c00502000000ba07040100000010636f6e74726163745f616464726573730a000000244b54314b6639633575664c574538394e347a425775427173454d61354732674644525372070401000000046e616d650a00000022566172696f75732053656375726974792054657374696e67205363656e6172696f730704010000000673796d626f6c0a00000005765365635407040100000008746f6b656e5f69640a00000004393030300704010000000a746f6b656e5f747970650a00000003464132`
  - Amount: `1_000`
  - Receiver: `KT1RZKsU9CD4siruhkxwzuwshwmmdXeM2ia4`
Successfully transfered, tx hash: `oom1A6jWR1xKoPoqFbjXVyyXcoNh8XSw22wHbBzZqSVgULyx17j`


### Getting ticket table value:
- Checking that token_proxy_tester received 1000 tickets:

In [8]:
get_tickets_count(ETHERLINK_ROLLUP_NODE_URL, ticket, token_proxy_tester.address)

1000

### Setup to make 1 call:
- TODO: need to find a way to simplify this setup, maybe reuse some logic from deployment, as this reuse the same parameters:

In [11]:
withdraw_amount = 1
calls_count = 3

# NOTE: this was the values in previous test:
# withdraw_amount = 10
# calls_count = 1
receiver = tezos_account
ticket = ticketer.read_ticket()
ticketer_address_bytes = bytes.fromhex(make_address_bytes(ticket.ticketer))
ticket_content_bytes = bytes.fromhex(ticket.content.to_bytes_hex())

result = token_proxy_tester.set_parameters(
    # TODO: try another scenario where ticket owner is another token_proxy_tester (ping-pong withdrawals)
    ticket_owner=token_proxy_tester.address,
    withdrawal_precompile=FA_WITHDRAWAL_PRECOMPILE,
    routing_info=make_withdrawal_routing_info(receiver, ticketer),
    amount=withdraw_amount,
    ticketer=ticketer_address_bytes,
    content=ticket_content_bytes,
    calls_count=calls_count
)

result

AttributeDict({'transactionHash': HexBytes('0xe00f0c1eaddb18adc6c91e32275e9f4e4f2a25518c86fd17500afd69a25036fc'),
 'transactionIndex': 0,
 'blockHash': HexBytes('0x03bcef747cf132305557499c909745e63f1989baab9fff24b396120df1ba63a0'),
 'blockNumber': 56945,
 'from': '0x7e6f6CCFe485a087F0F819eaBfDBfb1a49b97677',
 'to': '0x8f31Fa683c8DEdbE61fC772b2E0a29C35069FF9D',
 'cumulativeGasUsed': 2977395,
 'effectiveGasPrice': 1000000000,
 'gasUsed': 2977395,
 'logs': [],
 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

In [12]:
withdraw.callback(
    erc20_proxy_address=token_proxy_tester.address,
    tezos_side_router_address=ticketer.address,
    amount=1,
    # TODO: consider changing this to ticketer_address_bytes_hex ?
    ticketer_address_bytes=make_address_bytes(ticketer.address),
    # TODO: consider changing this to ticket_content_bytes_hex ?
    ticket_content_bytes=ticketer.read_content().to_bytes_hex(),
    receiver_address=get_address(tezos_account),
    withdraw_precompile=FA_WITHDRAWAL_PRECOMPILE,
    etherlink_private_key=ETHERLINK_PRIVATE_KEY,
    etherlink_rpc_url=ETHERLINK_RPC_URL,
)

Making FA withdrawal, ERC20 token: `0x8f31Fa683c8DEdbE61fC772b2E0a29C35069FF9D`:
  - Executor: `0x7e6f6CCFe485a087F0F819eaBfDBfb1a49b97677`
  - Etherlink RPC node: `https://etherlink.dipdup.net`
  - Withdrawal params:
      * Ticket owner: `0x8f31Fa683c8DEdbE61fC772b2E0a29C35069FF9D`
      * Receiver: `tz1ekkzEN2LB1cpf7dCaonKt6x9KVd9YVydc`
      * Router: `KT1TQiSYR6VhFHso5xkEYR3Fgg5Vsz8iRaK7`
      * Routing info: `0x0000d1b03118754a8e193b3e5d3a7cded976f4fd425601ce839f3e5187c0abec444760a58e950b675a334100`
      * Amount: `1`
      * Ticketer address bytes: `0x01ce839f3e5187c0abec444760a58e950b675a334100`
      * Content bytes: `0x0707000005090a000000c00502000000ba07040100000010636f6e74726163745f616464726573730a000000244b54314b6639633575664c574538394e347a425775427173454d61354732674644525372070401000000046e616d650a00000022566172696f75732053656375726974792054657374696e67205363656e6172696f730704010000000673796d626f6c0a00000005765365635407040100000008746f6b656e5f69640a000000043930303007040

'0xf5b8ce6311c779e127b504baf05623a0655cd4ed7c930a03d8ed15fc1d217f60'

- NOTE: need to wait until rollup processes transaction, not sure what is the best way to do this, probably simple time.sleep would work?

- checking that it has 11 tokens less (1_000 - 11 = 989)

In [13]:
get_tickets_count(ETHERLINK_ROLLUP_NODE_URL, ticket, token_proxy_tester.address)

996

In [16]:
ticket = ticketer.read_ticket(tezos_account)
ticket, _ = ticket.split(21)

opg = transfer_ticket(
    ticket=ticket,
    receiver=ticket_router_tester,
)

Transfering ticket:
  - Owner: `tz1ekkzEN2LB1cpf7dCaonKt6x9KVd9YVydc`
  - Ticketer address: `KT1TQiSYR6VhFHso5xkEYR3Fgg5Vsz8iRaK7`
  - Ticket content: `0x0707000005090a000000c00502000000ba07040100000010636f6e74726163745f616464726573730a000000244b54314b6639633575664c574538394e347a425775427173454d61354732674644525372070401000000046e616d650a00000022566172696f75732053656375726974792054657374696e67205363656e6172696f730704010000000673796d626f6c0a00000005765365635407040100000008746f6b656e5f69640a00000004393030300704010000000a746f6b656e5f747970650a00000003464132`
  - Amount: `21`
  - Receiver: `KT1RZKsU9CD4siruhkxwzuwshwmmdXeM2ia4`
Successfully transfered, tx hash: `oneJHc1JmnuL2747VZdrBtwdkfhDQeQZ1JZ1y8tPJuAfSaBjKAA`


- TODO: check that amount is expected, 1000

In [18]:
get_tickets_count(ETHERLINK_ROLLUP_NODE_URL, ticket, token_proxy_tester.address)

1014

### The following checked with 1000 > amount > 990 and it fails, but 990 works:

In [20]:
result = withdraw.callback(
    erc20_proxy_address=token_proxy_tester.address,
    tezos_side_router_address=ticketer.address,
    amount=1004,
    # TODO: consider changing this to ticketer_address_bytes_hex ?
    ticketer_address_bytes=make_address_bytes(ticketer.address),
    # TODO: consider changing this to ticket_content_bytes_hex ?
    ticket_content_bytes=ticketer.read_content().to_bytes_hex(),
    receiver_address=get_address(tezos_account),
    withdraw_precompile=FA_WITHDRAWAL_PRECOMPILE,
    etherlink_private_key=ETHERLINK_PRIVATE_KEY,
    etherlink_rpc_url=ETHERLINK_RPC_URL,
)

Making FA withdrawal, ERC20 token: `0x8f31Fa683c8DEdbE61fC772b2E0a29C35069FF9D`:
  - Executor: `0x7e6f6CCFe485a087F0F819eaBfDBfb1a49b97677`
  - Etherlink RPC node: `https://etherlink.dipdup.net`
  - Withdrawal params:
      * Ticket owner: `0x8f31Fa683c8DEdbE61fC772b2E0a29C35069FF9D`
      * Receiver: `tz1ekkzEN2LB1cpf7dCaonKt6x9KVd9YVydc`
      * Router: `KT1TQiSYR6VhFHso5xkEYR3Fgg5Vsz8iRaK7`
      * Routing info: `0x0000d1b03118754a8e193b3e5d3a7cded976f4fd425601ce839f3e5187c0abec444760a58e950b675a334100`
      * Amount: `1_004`
      * Ticketer address bytes: `0x01ce839f3e5187c0abec444760a58e950b675a334100`
      * Content bytes: `0x0707000005090a000000c00502000000ba07040100000010636f6e74726163745f616464726573730a000000244b54314b6639633575664c574538394e347a425775427173454d61354732674644525372070401000000046e616d650a00000022566172696f75732053656375726974792054657374696e67205363656e6172696f730704010000000673796d626f6c0a00000005765365635407040100000008746f6b656e5f69640a00000004393030300

- TODO: check expected ticket count is 0

In [22]:
get_tickets_count(ETHERLINK_ROLLUP_NODE_URL, ticket, token_proxy_tester.address)

7

- TODO: rewrite this:
- NEXT: deposit when contract tries to make withdrawal when there are no tickets, but deposit still should succeed because even if it failed the receiver should get tickets and receiver is the TokenProxyTester:

In [24]:
ticket = ticketer.read_ticket(tezos_account)
ticket, _ = ticket.split(15)

opg = transfer_ticket(
    ticket=ticket,
    receiver=ticket_router_tester,
)

Transfering ticket:
  - Owner: `tz1ekkzEN2LB1cpf7dCaonKt6x9KVd9YVydc`
  - Ticketer address: `KT1TQiSYR6VhFHso5xkEYR3Fgg5Vsz8iRaK7`
  - Ticket content: `0x0707000005090a000000c00502000000ba07040100000010636f6e74726163745f616464726573730a000000244b54314b6639633575664c574538394e347a425775427173454d61354732674644525372070401000000046e616d650a00000022566172696f75732053656375726974792054657374696e67205363656e6172696f730704010000000673796d626f6c0a00000005765365635407040100000008746f6b656e5f69640a00000004393030300704010000000a746f6b656e5f747970650a00000003464132`
  - Amount: `15`
  - Receiver: `KT1RZKsU9CD4siruhkxwzuwshwmmdXeM2ia4`
Successfully transfered, tx hash: `onpxBE4aotrsntkiemttPvQpoWHdHjWtXdzHDtfYnQ43T555XHE`


- TODO: check expected tickets is 15

In [25]:
get_tickets_count(ETHERLINK_ROLLUP_NODE_URL, ticket, token_proxy_tester.address)

7

## ?
- Scenario B: `TokenProxyTester` withdraws tickets twice during withdrawal, but it has not enough tickers
- setup:
    - tester originated with callsCount=0
    - deposit 10 tickets to the proxy from L1 via kernel
    - tester resetup to have callsCount=1, amount=10
- test:
    - user calls `fa_withdrawal_precompile` to withdraw 1 ticket from proxy
        * tester tries to withdraw 10 tickets more in the internal call
    - check that transaction failed
    - check that `fa_withdrawal_precompile` not broken after this failure

## ?
- Scenario C: `TokenProxyTester` withdraws tickets during deposit
- setup:
    - tester originated with callsCount=1, amount=11
- test:
    - deposit 10 tickets to the proxy from L1 via kernel
        * tester tries to withdraw 11 tickets and fails

## ?
- Scenario D: with successful withdrawal during deposit (withdraw less than deposit)
- TODO: consider making abstract test for deposit/withdrawal with different setups?
    - some kind of fuzz testing? parametrize all of the scenarios into one and check them all? and try to find more?